In [51]:
import os
import jieba
import numpy as np
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
# 停用词列表
stopwords = []

# 加载停用词列表
with open('../stopwords/cn_stopwords.txt', encoding='utf-8') as f:
    for line in f:
        stopwords.append(line.strip())

with open('../stopwords/hit_stopwords.txt', encoding='utf-8') as f:
    for line in f:
        stopwords.append(line.strip())

with open('../stopwords/baidu_stopwords.txt', encoding='utf-8') as f:
    for line in f:
        stopwords.append(line.strip())

with open('../stopwords/scu_stopwords.txt', encoding='utf-8') as f:
    for line in f:
        stopwords.append(line.strip())

In [33]:
# 读取train数据集
data = []
labels = []
for root, dirs, files in os.walk('./Chinese_Information_Retrieval_Task_Dataset/train'):
    print('dealing with {}'.format(root.split('/')[-1]))
    for file in files:
        with open(os.path.join(root, file), 'r', encoding='GB2312',errors='ignore') as f:
            content = f.read()
            # 中文分词
            words = jieba.lcut(content)
            # 过滤停用词和数字
            words = [word for word in words if word not in stopwords and not word.isdigit()]

            data.append(' '.join(words))
            labels.append(root.split('/')[-1])

dealing with train
dealing with 18
dealing with 9
dealing with 17
dealing with 8
dealing with 23
dealing with 14
dealing with 4
dealing with 6
dealing with 24
dealing with 3
dealing with 2
dealing with 22
dealing with 20
dealing with 12
dealing with 21
dealing with 7
dealing with 16
dealing with 13
dealing with 1
dealing with 25
dealing with 10
dealing with 15
dealing with 5
dealing with 11
dealing with 26
dealing with 19


In [34]:
test_data = data

In [35]:
# 检测是否data已包含全部的训练数据
len(data)

4411

In [37]:
# 使用TF-IDF算法将文本转换成向量
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(data)

In [40]:
tfidf_matrix

<4411x58141 sparse matrix of type '<class 'numpy.float64'>'
	with 663459 stored elements in Compressed Sparse Row format>

In [45]:
test_content = []
with open('./Chinese_Information_Retrieval_Task_Dataset/test.txt', 'r', encoding='GB2312', errors='ignore') as f:
    lines = f.readlines()
    for line in lines:
        test_content.append(line)
print(test_content)

['1: 中国女曲能否击败韩国圆梦\n', '2: 女垒锦标赛的会徽和吉祥物“丫丫”在南京首次亮相\n', '3: 男子十米中国队双保险失灵 加澳抢得冠亚军\n', '4: 中国体操在悉尼的辉煌与遗憾\n', '5: 棒球联赛第二阶段首回合 沪上闷热击溃猛虎\n', '6: 老帅盛赞国奥小将：具备亚洲顶级水平\n', '7: 全国山地车冠军赛第二站：马艳萍获女子越野赛桂冠\n', '8: 摩托罗拉世界名校赛艇对抗赛\n', '9: 网球女单开赛：塞莱斯首战轻取对手\n', '10: NBA：湖人主帅杰克逊支持科比']


In [44]:
# import re

# for i in range(len(test_content)):
#     tmp = test_content[i]
#     s = re.sub(r"\d+", "", tmp)
#     print(s)

: 中国女曲能否击败韩国圆梦

: 女垒锦标赛的会徽和吉祥物“丫丫”在南京首次亮相

: 男子十米中国队双保险失灵 加澳抢得冠亚军

: 中国体操在悉尼的辉煌与遗憾

: 棒球联赛第二阶段首回合 沪上闷热击溃猛虎

: 老帅盛赞国奥小将：具备亚洲顶级水平

: 全国山地车冠军赛第二站：马艳萍获女子越野赛桂冠

: 摩托罗拉世界名校赛艇对抗赛

: 网球女单开赛：塞莱斯首战轻取对手

: NBA：湖人主帅杰克逊支持科比


In [46]:
# 将查询文本转换为tf-idf向量
query_vector = []
for i in range(len(test_content)):
    tmp = vectorizer.transform([test_content[i]])
    query_vector.append(tmp)

In [49]:
# 计算查询文本与文本库中的每个文本的相似度得分
similarity_scores = cosine_similarity(query_vector[0], tfidf_matrix)
similarity_scores

array([[0., 0., 0., ..., 0., 0., 0.]])

In [53]:
# 对相似度得分进行排序，并返回排名前K的文本
top_k = 2
indices = np.argsort(similarity_scores)[0][::-1][:top_k]
for i, idx in enumerate(indices):
    print(f"Top {i+1} match: {data[idx]}")

Top 1 match: 
 
 中国台北 选手 获 两枚 跆拳道 铜牌 Olympic.163   
 
 　 　 赛程 　 比赛项目 　 百年 奥运 　 华夏 风流 　 奥运场馆 　 奥运 吉祥物 　 悉尼 风情 　 奥运 股市   　 .. 返回 首页   
 
 　 位置 网易 首页     奥运 首页     go2map   悉尼 战报 跆拳道   文章 来源   网易 社区     
 
 
 中国台北 选手 获 两枚 跆拳道 铜牌 
   
 
 　 　 刚刚 结束 跆拳道 争夺 第三名 比赛 中 中国台北 选手 黄志雄 夺得 男子 公斤 级 铜牌 中国台北 女选手 纪淑茹 女子 公斤 级 铜牌   
 
   　 　   
 
 
 　 我要 发表 评论       
 
 　 相关 文章   　 热力 推荐     
 
 快讯 澳大利亚 选手 夺得 女子 公斤 级 跆拳道 金牌   
 盛泽田 无怨无悔   
 盛泽田 退役   
 美国 选手 获 公斤 级 古典式 摔跤 金牌   
 古巴 选手 获 男子 古典式 摔跤 公斤 级 金牌       
 
 奥运 股市   
 马家军 梦断 悉尼   
 战地 日记   
 奥运 论坛   
 奥运 精彩 瞬间   
 关闭 窗口   
 
 About   Netease     公司简介     网站 导航     联系方法     广告 服务     招聘 信息     客户服务     相关 法律 
 aoyunservice staff.163 
 网易公司 版权所有   copy            
Top 2 match: 
 
 釜山 亚运会   
 
 新闻 中心   釜山 评论   金牌 人物   赛程   资料   图酷   反 兴奋剂   直播   短信   有奖 活动           
 
 体育 在线     釜山 亚运会       
 
   花剑 王 海滨 战胜 金相勋 　 夺得 亚运会 首金 
 
         人民网 北京 月 日讯 　 刚刚 结束 男子 花剑 决赛 中 中国 队员 王 海滨 不利 情况 反败为胜 第二局 比赛 中以 战胜 金相勋 勇夺 男子 花剑 金牌 中国队 夺取 第一枚 金牌 亚运会 第一枚 金牌 
 
         王